In [3]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

VETS_CSV = "/Users/martinsvitek/layered-populate-data-pool-da/layered-populate-data-pool-da/vet_clinics/sources/vet_clinics/sources/vets_osm_berlin_cleaned_for_db.csv"
LOR_GEOJSON = "/Users/martinsvitek/layered-populate-data-pool-da/layered-populate-data-pool-da/vet_clinics/sources/vet_clinics/sources/lor_ortsteile.geojson"
OUT_CSV = "/Users/martinsvitek/layered-populate-data-pool-da/layered-populate-data-pool-da/vet_clinics/sources/vets_with_districts_neighborhoods.csv"

# 1) Load vets and build points
v = pd.read_csv(VETS_CSV)
v["latitude"]  = pd.to_numeric(v["latitude"], errors="coerce")
v["longitude"] = pd.to_numeric(v["longitude"], errors="coerce")
v = v.dropna(subset=["latitude","longitude"]).copy()

v_gdf = gpd.GeoDataFrame(
    v,
    geometry=gpd.points_from_xy(v["longitude"], v["latitude"]),
    crs="EPSG:4326"
)

# 2) Load LOR polygons and pick the actual columns you have
lor = gpd.read_file(LOR_GEOJSON)

# Ensure CRS matches
if lor.crs is None:
    lor = lor.set_crs(4326, allow_override=True)
elif lor.crs.to_epsg() != 4326:
    lor = lor.to_crs(4326)

# Find name columns (your file shows BEZIRK and OTEIL)
def first_present(cands, df):
    for c in cands:
        if c in df.columns:
            return c
    return None

district_name_col = first_present(["BEZIRK", "Bezirk", "district", "DISTRICT"], lor)
neigh_name_col    = first_present(["OTEIL", "Ortsteil", "ORTSTEIL", "neighbourhood", "NEIGHBOURHOOD"], lor)

if not district_name_col or not neigh_name_col:
    raise KeyError(f"Couldn’t find district/neighbourhood name columns in GeoJSON. "
                   f"Columns present: {sorted(lor.columns)}")

lor_min = lor[[district_name_col, neigh_name_col, "geometry"]].copy()
lor_min.rename(columns={district_name_col: "district", neigh_name_col: "neighbourhood"}, inplace=True)

# If official numeric IDs exist, pick them up; else create IDs from names
district_id_col = first_present(
    ["Bezirk_ID","BEZIRKS_ID","BEZIRKSNR","BEZIRK_ID","Schluessel_gesamt","BEZIRK_NR","district_id"],
    lor
)
neigh_id_col = first_present(
    ["Ortsteil_ID","ORTSTEIL_ID","ORTSTEIL_NR","Ortsteil_Nr","neighbourhood_id"],
    lor
)

if district_id_col and district_id_col in lor.columns:
    lor_min["district_id"] = lor[district_id_col].values
else:
    lor_min["district_id"] = lor_min["district"].astype("category").cat.codes + 1

if neigh_id_col and neigh_id_col in lor.columns:
    lor_min["neighbourhood_id"] = lor[neigh_id_col].values
else:
    lor_min["neighbourhood_id"] = lor_min["neighbourhood"].astype("category").cat.codes + 1

# 3) Spatial join (point-in-polygon). If many points lie on borders, try predicate="intersects".
joined = gpd.sjoin(v_gdf, lor_min, how="left", predicate="within")
if "index_right" in joined.columns:
    joined.drop(columns=["index_right"], inplace=True)

# 4) Guarantee the columns exist (derive IDs from names if still missing)
if "district" in joined.columns and "district_id" not in joined.columns:
    joined["district_id"] = joined["district"].astype("category").cat.codes + 1
if "neighbourhood" in joined.columns and "neighbourhood_id" not in joined.columns:
    joined["neighbourhood_id"] = joined["neighbourhood"].astype("category").cat.codes + 1

# 5) Save CSV (only columns that actually exist)
base_cols = list(v.columns)  # original vets columns
extra_cols = ["district_id", "district", "neighbourhood_id", "neighbourhood"]
present_extras = [c for c in extra_cols if c in joined.columns]
out_cols = base_cols + present_extras

os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
joined[out_cols].to_csv(OUT_CSV, index=False, encoding="utf-8")

print(f"Saved: {OUT_CSV} ({len(joined)} rows)")
print("Added columns present:", present_extras)


Saved: /Users/martinsvitek/layered-populate-data-pool-da/layered-populate-data-pool-da/vet_clinics/sources/vets_with_districts_neighborhoods.csv (178 rows)
Added columns present: ['district_id', 'district', 'neighbourhood_id', 'neighbourhood']
